# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [32]:
# Drop tables and create 'fresh' tables
# by running the 'create_tables.py' script

!python create_tables.py

Tables Dropped If Exists.
Tables All Created.


In [33]:
import os
import glob
import psycopg2
import pandas as pd
from datetime import datetime
from sql_queries import *
from prettytable import PrettyTable

# Suppress SettingWithCopyWarning in Pandas
pd.options.mode.chained_assignment = None

In [34]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [35]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, '*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))

    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [36]:
filepath = get_files('data/')
song_files = get_files('data/song_data')
log_files = get_files('data/log_data')

In [37]:
# Concat all songs data
dfs = []

for file in song_files:
    x = pd.read_json(file, lines=True)
    dfs.append(x)

# Concat all songs data

df_songs = pd.concat(dfs)
df_songs.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
0,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984
0,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
0,1,AR10USD1187B99F3F1,NaN,NaN,"Burlington, Ontario, Canada",Tweeterfriendly Music,SOHKNRJ12A6701D1F8,Drop of Rain,189.57016,0
0,1,ARGSJW91187B9B1D6B,35.21962,-80.01955,North Carolina,JennyAnyKind,SOQHXMF12AB0182363,Young Boy Blues,218.77506,0


In [38]:
# song data

# songs - songs in music database
# song_id, title, artist_id, year, duration

song_data = df_songs[['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data = song_data.drop_duplicates('song_id')
song_data = song_data.set_index('song_id')
song_data = song_data.to_records()
song_data = list(song_data)

# convert numpy numeric to python native numeric. otherwise psycopg2 wont accept
song_data = [(x[0], x[1], x[2], int(x[3]), float(x[4])) for x in song_data]

In [39]:
# artist data

# artist_id, name, location, latitude, longitude

artist_data = df_songs[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data = artist_data.drop_duplicates('artist_id')
artist_data = artist_data.set_index('artist_id')
artist_data = artist_data.to_records()
artist_data = list(artist_data)

artist_data = [(x[0], x[1], x[2], float(x[3]), float(x[4])) for x in artist_data]

In [40]:
# Concat all event logs data
dfl = []

for file in log_files:
    x = pd.read_json(file, lines=True)
    dfl.append(x)

# Concat

df_logs = pd.concat(dfl)
df_logs.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


In [41]:
# df logs data subset to song playing event

df_logs = df_logs[df_logs['page'] == 'NextSong']

In [42]:
# load user table

# user_id, first_name, last_name, gender, level

user_df = df_logs[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df = user_df.drop_duplicates('userId', keep='last')
user_df = user_df.set_index('userId')
user_data = user_df.to_records()
user_data = list(user_data)

print(len(user_data))

129


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [43]:
# Spot Check
song_data[0]

('SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [44]:
# Insert all songs data records
try:
    for row in song_data:
        # print(row)
        cur.execute(song_table_insert, row)
        conn.commit()
    print('Insert song rows done')

except Exception as e:
    conn.rollback()
    print('Error in inserting song data. ')
    print(e)

Insert song rows done


In [45]:
# Read from db, check data inserted

cur.execute("select * from songs")
conn.commit()

t = PrettyTable(['song_id', 'title', 'artist_id', 'year', 'duration'])

for (song_id, title, artist_id, year, duration) in cur:
    t.add_row([song_id, title, artist_id, year, duration])

print(t)

+--------------------+------------------------------------------------------+--------------------+------+-----------+
|      song_id       |                        title                         |     artist_id      | year |  duration |
+--------------------+------------------------------------------------------+--------------------+------+-----------+
| SONHOTT12A8C13493C |                   Something Girls                    | AR7G5I41187FB4CE6C | 1982 | 233.40363 |
| SOIAZJW12AB01853F1 |                      Pink World                      | AR8ZCNI1187B9A069B | 1984 | 269.81832 |
| SOFSOCN12A8C143F5D |                    Face the Ashes                    | ARXR32B1187FB57099 | 2007 | 209.60608 |
| SOHKNRJ12A6701D1F8 |                     Drop of Rain                     | AR10USD1187B99F3F1 |  0   | 189.57016 |
| SOQHXMF12AB0182363 |                   Young Boy Blues                    | ARGSJW91187B9B1D6B |  0   | 218.77506 |
| SOCIWDW12A8C13D406 |                      Soul Deep   

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [46]:
# artist_data =
# artist_data

artist_data[3]


('AR10USD1187B99F3F1',
 'Tweeterfriendly Music',
 'Burlington, Ontario, Canada',
 nan,
 nan)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [47]:
# cur.execute(artist_table_insert, artist_data[4])
# conn.commit()

In [48]:
# Insert all artist data records

try:
    print('Inserting artist rows.')
    for row in artist_data:
        # print(row)
        cur.execute(artist_table_insert, row)
        conn.commit()

    print('Insert artist rows done')

except Exception as e:
    conn.rollback()
    print('Error in inserting artist data. ')
    print(e)

Inserting artist rows.
Insert artist rows done


In [49]:
# Read from db, check data inserted

cur.execute("select * from artists")
conn.commit()

t = PrettyTable(['artist_id', 'name', 'location', 'latitude', 'longitude'])

for (artist_id, name, location, latitude, longitude) in cur:
    t.add_row([artist_id, name, location, latitude, longitude])

print(t)

+--------------------+------------------------------------------------------------------------------------------------+-------------------------------+----------+--------------------+
|     artist_id      |                                              name                                              |            location           | latitude |     longitude      |
+--------------------+------------------------------------------------------------------------------------------------+-------------------------------+----------+--------------------+
| AR7G5I41187FB4CE6C |                                            Adam Ant                                            |        London, England        |   NaN    |        NaN         |
| AR8ZCNI1187B9A069B |                                        Planet P Project                                        |                               |   NaN    |        NaN         |
| ARXR32B1187FB57099 |                                              Gob         

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [50]:
# Type cast and extract specific datetime info

t = pd.to_datetime(df_logs['ts'], unit='ms')

time_df = pd.DataFrame({
    'ts': t,
    'hour': t.dt.hour,
    'day': t.dt.day,
    'week': t.dt.isocalendar().week,
    'month': t.dt.month,
    'year': t.dt.year,
    'weekday': t.dt.weekday

})

time_df.head()

,ts,hour,day,week,month,year,weekday
0,2018-11-11 02:33:56.796,2,11,45,11,2018,6
1,2018-11-11 02:36:10.796,2,11,45,11,2018,6
2,2018-11-11 02:40:34.796,2,11,45,11,2018,6
4,2018-11-11 04:36:13.796,4,11,45,11,2018,6
5,2018-11-11 04:36:46.796,4,11,45,11,2018,6


In [51]:
# Insert time records

try:
    print('Inserting time records.')

    for _, row in time_df.iterrows():
        cur.execute(time_table_insert, tuple(row))
        conn.commit()

    print('Insert time rows done.')

except Exception as e:
    conn.rollback()
    print('Error in inserting time data. ')
    print(e)


Inserting time records.
Insert time rows done.


In [52]:
# Read from db, check data inserted

cur.execute("select * from time limit 500")
conn.commit()

t = PrettyTable(['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'])

for (start_time, hour, day, week, month, year, weekday) in cur:
    t.add_row([start_time, hour, day, week, month, year, weekday])

print(t)

# start_time, hour, day, week, month, year, weekday

+----------------------------+------+-----+------+-------+------+---------+
|         start_time         | hour | day | week | month | year | weekday |
+----------------------------+------+-----+------+-------+------+---------+
| 2018-11-11 02:33:56.796000 |  2   |  11 |  45  |   11  | 2018 |    6    |
| 2018-11-11 02:36:10.796000 |  2   |  11 |  45  |   11  | 2018 |    6    |
| 2018-11-11 02:40:34.796000 |  2   |  11 |  45  |   11  | 2018 |    6    |
| 2018-11-11 04:36:13.796000 |  4   |  11 |  45  |   11  | 2018 |    6    |
| 2018-11-11 04:36:46.796000 |  4   |  11 |  45  |   11  | 2018 |    6    |
| 2018-11-11 09:56:28.796000 |  9   |  11 |  45  |   11  | 2018 |    6    |
| 2018-11-11 10:00:47.796000 |  10  |  11 |  45  |   11  | 2018 |    6    |
| 2018-11-11 10:04:07.796000 |  10  |  11 |  45  |   11  | 2018 |    6    |
| 2018-11-11 10:08:28.796000 |  10  |  11 |  45  |   11  | 2018 |    6    |
| 2018-11-11 10:12:00.796000 |  10  |  11 |  45  |   11  | 2018 |    6    |
| 2018-11-11

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [53]:
try:
    print('Inserting user records.')
    for row in user_data:
        # print(row)
        cur.execute(user_table_insert, row)
        conn.commit()

    print('Insert users done.')

except Exception as e:
    conn.rollback()
    print('Error in inserting users data. ')
    print(e)



Inserting user records.
Insert users done.


In [54]:
# Read from user table check

cur.execute("select * from users")
conn.commit()

t = PrettyTable(['user_id', 'first_name', 'last_name', 'gender', 'level'])

for (user_id, first_name, last_name, gender, level) in cur:
    t.add_row([user_id, first_name, last_name, gender, level])

print(t)

# user_id, first_name, last_name, gender, level

+---------+------------+-----------+--------+-------+
| user_id | first_name | last_name | gender | level |
+---------+------------+-----------+--------+-------+
|    56   |   Cienna   |  Freeman  |   F    |  free |
|    59   |    Lily    |   Cooper  |   F    |  free |
|    20   |   Aiden    |  Ramirez  |   M    |  paid |
|    72   |   Hayden   |   Brock   |   F    |  paid |
|    48   |   Marina   |   Sutton  |   F    |  free |
|    3    |   Isaac    |   Valdez  |   M    |  free |
|    98   |   Jordyn   |   Powell  |   F    |  free |
|    79   |   James    |   Martin  |   M    |  free |
|    4    |   Alivia   |  Terrell  |   F    |  free |
|    22   |    Sean    |   Wilson  |   F    |  free |
|    78   |   Chloe    |    Roth   |   F    |  free |
|    64   |   Hannah   |  Calhoun  |   F    |  free |
|    47   |   Kimber   |   Norris  |   F    |  free |
|    99   |    Ann     |   Banks   |   F    |  free |
|    38   |   Gianna   |   Jones   |   F    |  free |
|    5    |   Elijah   |   D

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [55]:
# songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent

df = df_logs.copy()

In [56]:
# songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
try:
    print('Inserting songplay records.')

    for index, row in df.iterrows():

        start_time = pd.to_datetime(row.ts, unit='ms') # typecast to timestamp
        user_id = row.userId
        level = row.level
        song_title = row.song
        artist_name = row.artist
        session_id = row.sessionId
        location = row.location
        user_agent = row.userAgent
        duration = row.length

        # parameters data from select song id and artist id
        params = (song_title, artist_name, duration)

        # get songid and artistid from song and artist tables
        cur.execute(song_select, params)
        results = cur.fetchone()
        # print(f"results: {results}")

        if results:
            song_id, artist_id = results
        else:
            song_id, artist_id = None, None

        # insert songplay record
        songplay_data = (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()

    print("Insert songplay records done")


except Exception as e:
    conn.rollback()
    print('Error in inserting songplay data. ')
    print(e)

Inserting songplay records.
Insert songplay records done


In [57]:
# Read from songplays data

cur.execute("select * from songplays limit 500")
conn.commit()

t = PrettyTable(
    ['songplay_id', 'start_time', 'user_id', 'level', 'song_id', 'artist_id', 'session_id', 'location', 'user_agent'])

for (songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) in cur:
    t.add_row([songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent])

print(t)

# songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent

+-------------+----------------------------+---------+-------+---------+-----------+------------+------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+
| songplay_id |         start_time         | user_id | level | song_id | artist_id | session_id |                    location                    |                                                                 user_agent                                                                |
+-------------+----------------------------+---------+-------+---------+-----------+------------+------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+
|      1      | 2018-11-11 02:33:56.796000 |    69   |  free |   None  |    None   |    455     |  Philadelphia-Camden-Wilmington, PA-NJ-DE

In [58]:
# test_data = ('Love Story', 'Taylor Swift')
#
# cur.execute("""
# select song_id, artist_id
# from songs
# inner join artists
# using (artist_id)
# where title = %s and name = %s
# """, test_data)
# conn.commit()
#
# res = cur.fetchone()
# print(res)
#
# # res = cur.fetchone()
#
# # t = PrettyTable(['song_id', 'artist_id'])
# # for (a, b) in cur:
# #     t.add_row([a, b])
# #
# # print(t)

# Close Connection to Sparkify Database

In [59]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.